<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Trabalho_2_Sentimento_Noriaki_Suda_Wakabayashi_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-tweets
*   Selecione a opção "Adicionar Atalho ao Drive"

In [1]:
!pip install mrjob nltk twython

     |████████████████████████████████| 439 kB 5.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/twitter_cs"

negative-words.txt  sample_clean_larger.csv	twcs_clean.csv
positive-words.txt  sample_clean_larger.csv.gz	twcs_clean.csv.gz
sample_clean.csv    sample.csv			twcs.csv


In [4]:
!cat "/content/drive/My Drive/twitter_cs/sample_clean.csv" | head

tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡,119236,
119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,"@105835 Your business means a lot to us. Please DM your name  zip code and additional details about your concern. ^RR https://t.co/znUu1VJn9r",,119239
119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm sure you won't! Because you don't have to!,119238,
119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,"@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 option 1  4  3 (Leave a message) to request a call back",119241,119242
119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've tried leaving a voicemail several times in the past week https://t.co/NxVZj

# Ver porque nao esta saindo resultado pelo logger de warning

In [48]:
%%file trab2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import TextProtocol
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re
import logging

class Sentiment(MRJob):

  OUTPUT_PROTOCOL = TextProtocol  
  
  def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
            mapper=self.mapper,
            combiner=self.combiner,
            reducer=self.reducer),
            MRStep(reducer=self.reducer_order)
        ]

  def mapper_init(self):
    nltk.download('vader_lexicon', quiet=True)
    self.sentiment = SentimentIntensityAnalyzer()

  def mapper(self, _, line):
    try:

      tweet_id, author_id, inbound, created_at, text, response_tweet_id, in_response_to_tweet_id = line.split(",")

      author_id = author_id.lower().strip()
      flag_author = bool(re.match("^[0-9]+$", author_id))

      if inbound == "True":
        raise ValueError("Inbound") 

      elif author_id == "author_id":
        raise ValueError("Primeira Linha")

      elif flag_author == True:
        raise ValueError("Author")

      polaridade = self.sentiment.polarity_scores(text)['compound']

      yield author_id, (polaridade, 1)

    except ValueError as error:
      logging.warning(error.args)
      logging.warning("linha descartada: "+line)
      pass

  def combiner(self, key, values):
    try:
      polaridade_sum = 0
      quantidade = 0
      for value in values:
        polaridade_sum += value[0]
        quantidade += value[1]
    
      yield key, (polaridade_sum, quantidade)
    
    except:
      logging.warning("Deu ruim Combiner")     
      pass

  def reducer(self, key, values):
    try:
      polaridade_sum = 0
      quantidade = 0
      for value in values:
        polaridade_sum += value[0]
        quantidade += value[1]
      # if(polaridade_sum/quantidade) >= 0:
      #   quantity = "{:.4f}".format(polaridade_sum/quantidade).zfill(6)
      # else:
      quantity = polaridade_sum/quantidade
      yield None, (quantity, key)

    except:
      logging.warning("Deu ruim reducer")     
      pass
  
  def reducer_order(self, quantity, keys):
    try:
      for quantity, key in sorted(keys):
        yield key, "{:.4f}".format(quantity).zfill(6)
    except:
      logging.warning("Deu ruim reducer 2")

if __name__ == '__main__':
    logging.basicConfig(filename="log_trabalho2.txt",level=logging.INFO)
    Sentiment.run()

Overwriting trab2.py


In [49]:
!python trab2.py "/content/drive/My Drive/twitter_cs/sample_clean.csv" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/trab2.root.20211107.011107.727588
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/trab2.root.20211107.011107.727588/output
Streaming final output from /tmp/trab2.root.20211107.011107.727588/output...
ask_spectrum	-0.2732
comcastcares	-0.1217
virgintrains	0.1710
british_airways	0.2015
sprintcare	0.2263
upshelp	0.2263
chasesupport	0.3182
tesco	0.4806
spotifycares	0.5123
applesupport	0.5644
southwestair	0.6114
hpsupport	0.6369
o2	0.7783
Removing temp directory /tmp/trab2.root.20211107.011107.727588...


In [ ]:
# Sanity Check pra ver se esta agrupando certo
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon', quiet=True)
analyzer = SentimentIntensityAnalyzer()

df = pd.read_csv("/content/drive/My Drive/twitter_cs/sample_clean.csv" )

df["test"] = df.author_id.str.match("^[0-9]+$")

df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]

df[df["test"]==False].filter(["author_id", "compound"]).groupby("author_id")["compound"].mean().add(1).sort_values()

author_id
Ask_Spectrum       0.726800
comcastcares       0.878300
VirginTrains       1.171025
British_Airways    1.201500
UPSHelp            1.226300
sprintcare         1.226300
ChaseSupport       1.318200
Tesco              1.480625
SpotifyCares       1.512325
AppleSupport       1.564415
SouthwestAir       1.611400
HPSupport          1.636900
O2                 1.778300
Name: compound, dtype: float64

In [50]:
!cat log_trabalho2.txt

INFO:mrjob.conf:No configs found; falling back on auto-configuration
INFO:mrjob.runner:Creating temp directory /tmp/trab2.root.20211107.003107.248326
INFO:mrjob.sim:Running step 1 of 2...
INFO:mrjob.sim:Running step 2 of 2...
INFO:mrjob.runner:job output is in /tmp/trab2.root.20211107.003107.248326/output
INFO:mrjob.runner:Streaming final output from /tmp/trab2.root.20211107.003107.248326/output...
INFO:mrjob.runner:Removing temp directory /tmp/trab2.root.20211107.003107.248326...
INFO:mrjob.conf:No configs found; falling back on auto-configuration
INFO:mrjob.runner:Creating temp directory /tmp/trab2.root.20211107.003403.391013
INFO:mrjob.sim:Running step 1 of 2...
INFO:mrjob.sim:Running step 2 of 2...
INFO:mrjob.runner:job output is in /tmp/trab2.root.20211107.003403.391013/output
INFO:mrjob.runner:Streaming final output from /tmp/trab2.root.20211107.003403.391013/output...
INFO:mrjob.runner:Removing temp directory /tmp/trab2.root.20211107.003403.391013...
INFO:mrjob.conf:No configs f

In [51]:
!ls

drive  log_trabalho2.txt  sample_data  trab2.py


In [52]:
!rm log_trabalho2.txt

In [53]:
!python trab2.py "/content/drive/My Drive/twitter_cs/sample_clean_larger.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/trab2.root.20211107.011229.498309
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/trab2.root.20211107.011229.498309/output
Streaming final output from /tmp/trab2.root.20211107.011229.498309/output...
jackbox	0.0000
yahoocare	0.0175
nationalrailenq	0.0204
kfc_uki_help	0.0440
greggsofficial	0.0455
mtnc_care	0.0573
virginamerica	0.0705
tacobellteam	0.0713
londonmidland	0.0846
pandorasupport	0.0927
sw_help	0.0942
gwrhelp	0.0982
virgintrains	0.1027
virginatlantic	0.1338
safaricom_care	0.1571
askvisa	0.1717
chipotletweets	0.1730
verizonsupport	0.1940
dunkindonuts	0.2022
amazonhelp	0.2028
uscellularcares	0.2058
argoshelpers	0.2067
askrobinhood	0.2154
xboxsupport	0.2189
nikesupport	0.2245
sainsburys	0.2304
spotifycares	0.2370
atviassist	0.2412
tfl	0.2426
askpapajohns	0.2443
askplaystation	0.2459
boostcare	0.2464
idea_cares	0.2560
jetblue	0.2560
postmates_

In [54]:
!python trab2.py "/content/drive/My Drive/twitter_cs/twcs_clean.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/trab2.root.20211107.011250.973164
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/trab2.root.20211107.011250.973164/output
Streaming final output from /tmp/trab2.root.20211107.011250.973164/output...
kfc_uki_help	-0.0501
nationalrailenq	-0.0083
sw_help	-0.0044
londonmidland	0.0211
tacobellteam	0.0289
gwrhelp	0.0517
jackbox	0.0586
mtnc_care	0.0867
greggsofficial	0.1055
virgintrains	0.1225
askvisa	0.1411
uscellularcares	0.1572
yahoocare	0.1655
verizonsupport	0.1689
askseagate	0.1746
askdsc	0.1766
virginatlantic	0.1810
safaricom_care	0.1832
argoshelpers	0.1906
nikesupport	0.1906
postmates_help	0.1979
amazonhelp	0.2007
chipotletweets	0.2066
tfl	0.2147
virginamerica	0.2218
jetblue	0.2242
pandorasupport	0.2327
nortonsupport	0.2331
askplaystation	0.2408
british_airways	0.2422
o2	0.2425
hulu_support	0.2440
idea_cares	0.2469
spotifycares	0.2474
pearsonsupp

In [ ]:
drive.flush_and_unmount()